In [ ]:
# run stopwatch
from tools import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load libraries, functions, palette, theme

In [ ]:
%run _libraries.ipynb

In [ ]:
%run _functions.ipynb

In [ ]:
save_files = 'files/Section4-Linear-models-research'

In [ ]:
save_img = 'docs/img/Section4-Linear-models-research'

In [ ]:
session_name = 'Section4-Linear-models-research'

# Section IV. Linear Models Comparing

## Load Saved Section if exists

## Load Data

In [ ]:
# dicts
datasets_dict = loadit(
    'datasets_dict', dir='Section3-Feature-selection-and-Preprocessing')
features_dict = loadit(
    'features_dict', dir='Section3-Feature-selection-and-Preprocessing')
simulation_datasets_dict = loadit(
    'simulation_datasets_dict', dir='Section3-Feature-selection-and-Preprocessing')
estimators_dict = loadit(
    'estimators_dict', dir='Section3-Feature-selection-and-Preprocessing')
evaluation_dict = loadit(
    'evaluation_dict', dir='Section3-Feature-selection-and-Preprocessing')
# datasets
train = datasets_dict['train'].copy()
train_cv = datasets_dict['train_cv'].copy()
# features
features = features_dict['features']
numeric = features_dict['numeric']
ordinal = features_dict['ordinal']
categorical = features_dict['categorical']
categorical_transform = features_dict['categorical_transform']
factor = features_dict['factor']
target = features_dict['target']

## Elastic Net regularization effects

### Datasets

In [ ]:
X = train_cv[features].copy()
y = train_cv[target]

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
y.head()

### Preprocessors

In [ ]:
encoder = OrdinalEncoder(
    encoding_method='ordered',
    variables=categorical_transform,
    missing_values='ignore',
    unseen='encode'
)

In [ ]:
scaler = StandardScaler()

### Common Hyperparameters

In [ ]:
alphas = np.logspace(-3, 3, 30)
n_folds = 5

In [ ]:
fig = plt.figure(figsize=(8, 2.5))
plt.plot(alphas)
# plt.grid(False)
plt.title('Alphas logspace')
axis_rstyle(
    xticks=[0, 30, 5],
    yticks=[0, 1000, 200])
plt.ylabel('Alpha')
plt.show()

savefig('alphas_logspace', save_img)

### GridSearch

In [ ]:
enet_cv_pipeline = Pipeline([
    ('encoder', encoder),
    ('scaler', scaler),
    ('estimator', ElasticNet(max_iter=100000))
])

In [ ]:
enet_cv_pipeline

In [ ]:
enet_params = [{
    'estimator__alpha': alphas
}]

In [ ]:
enet_cv = GridSearchCV(
    estimator=enet_cv_pipeline,
    param_grid=enet_params,
    scoring='neg_root_mean_squared_error',
    cv=n_folds,
    refit=False
)

In [ ]:
st = stopwatch.start()
enet_cv.fit(X, y)
print(f'Execution time: {stopwatch.stop(st)}')

In [ ]:
enet_cv_best_score = abs(enet_cv.best_score_)
enet_cv_best_score

In [ ]:
enet_cv_best_params = {}

for key in enet_cv.best_params_.keys():
    new_key = key.lstrip('estimator')
    new_key = new_key.lstrip('__')
    enet_cv_best_params[new_key] = enet_cv.best_params_[key]
    
enet_cv_best_params

In [ ]:
enet_cv_best_idx = enet_cv.best_index_
enet_cv_best_idx

In [ ]:
enet_results = pd.DataFrame(enet_cv.cv_results_)

In [ ]:
enet_results.head()

In [ ]:
param_cols = [
    'param_estimator__alpha'
]
round_list = [4, 1]

In [ ]:
# add params compact column
enet_results = cv_results_params_transform(
    enet_results, param_cols, round_list)

In [ ]:
# abs all numeric
enet_results = enet_results.apply(
    lambda x: x.abs() if x.dtype.kind in 'iufc' else x)

In [ ]:
loc_best_idx = enet_results['mean_test_score']==enet_cv_best_score
best_score_idx = enet_results[loc_best_idx].index.item()
best_score_idx

In [ ]:
vars_plot = [
    'parameters',
    'split0_test_score', 'split1_test_score', 'split2_test_score',
    'split3_test_score', 'split4_test_score'
]
cv_results_prep = pd.melt(
    enet_results, id_vars='parameters',
    value_vars=vars_plot, value_name='rmse')
cv_results_prep.head()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 2.5))

sns.lineplot(
    data=cv_results_prep,
    x='parameters',
    y='rmse',
    err_kws=({'alpha': 0.1})
)
sns.scatterplot(
    data=enet_results[enet_results['rank_test_score']==1],
    x='parameters',
    y='mean_test_score',
    s=50, color=palette[1], facecolor='none',
    ec=palette[1], linewidth=1.5, zorder=10, alpha=0.9
)
plt.xlabel(str.capitalize('Alpha'))
plt.ylabel('MRSE')
# ax.set_title('Elastic Net: regularization effect')
ax.tick_params(axis='x', rotation=60)
# ax.xaxis.set_major_locator(matplotlib.ticker.AutoLocator())
axis_rstyle()


plt.grid(False)
plt.show()

savefig('regularization', save_img)

### ELASTIC NET Model

In [ ]:
enet_regressor = ElasticNet(
    **enet_cv_best_params,
    random_state=seed
)

In [ ]:
enet = Pipeline([
    ('encoder', encoder),
    ('scaler', scaler),
    ('estimator', enet_regressor)
])

In [ ]:
enet.fit(X,y)

In [ ]:
enet_summary = pd.DataFrame({
    'feature': enet.feature_names_in_,
    'coeff': enet.named_steps['estimator'].coef_,
    'intercept': enet.named_steps['estimator'].intercept_
}).sort_values('coeff', key=abs, ascending=False)

In [ ]:
enet_summary[enet_summary['coeff'] != 0].shape

In [ ]:
enet_summary.head(10)

### Features Research with Elastic Net

In [ ]:
X_fr = X.copy()

In [ ]:
pipe = Pipeline([
    ('encoder', encoder),
    ('scaler', scaler),
])

In [ ]:
X_fr[features] = pipe.fit_transform(X_fr[features], y)

In [ ]:
st = stopwatch.start()
results_sim_dict = simulation_enet_features(X_fr, y, alphas)
print(f'Execution time: {stopwatch.stop(st)}')

In [ ]:
elnet_features_df = pd.DataFrame(results_sim_dict)

In [ ]:
# save the list of lists with chosen features by enet in variable
features_chosen = elnet_features_df['features_list'].copy()

In [ ]:
# modificate column 'features_list': list of strings -> string
# for more aesthetic view in df
elnet_features_df['features_list'] = \
    elnet_features_df['features_list'].apply(lambda x: ' , '.join(x))

In [ ]:
elnet_features_df[elnet_features_df['features_num'] > 0]

In [ ]:
alpha_log = np.log(elnet_features_df['alpha'])

In [ ]:
xup_labels = elnet_features_df['features_num'].values
ticks = [int(i) for i in np.arange(-7, 7.1, 1)]

fig, ax = plt.subplots(figsize=(8, 2.5))
# f = plt.figure(figsize=(7, 2.5))
# plt.title('Features by Elastic Net', loc='left', pad=12)
plt.scatter(
    x=alpha_log, y=elnet_features_df['score'],
    ec='face', lw=0, s=10, alpha=0.95)
plt.xlabel('log(ALPHA)')
plt.ylabel('RMSE')
plt.xlim(-7.25, 7.25)
plt.ylim(0.08, 0.40)
plt.xticks(ticks=ticks, labels=ticks)
plt.grid(False)

axis_rstyle()
# top xaxis
xaxis_top = plt.twiny()
xaxis_top.set_xlim(-7.25, 7.25)
xaxis_top.set_xticks(ticks=alpha_log)
xaxis_top.spines['top'].set_position(('outward', 5))
xaxis_top.tick_params(left=False, bottom=False, top=False)
xaxis_top.set_xticklabels(labels=xup_labels, weight='normal', fontsize=8)
xaxis_top.grid(False)
xaxis_top.spines['left'].set_visible(False)
xaxis_top.spines['bottom'].set_visible(False)
# vertical red liness
plt.axvline(
    -1.65, ymin=0.02, ymax=0.98, linestyle='--',
    lw=1, color=palette[1], alpha=0.95)
plt.axvline(
    -3.55, ymin=0.02, ymax=0.98, linestyle='--',
    lw=1, color=palette[1], alpha=0.95)
ax.spines['bottom'].set_position(('outward', 12))

plt.show()

savefig('features_plot', save_img)

In [ ]:
features_chosen = \
    [element for sublist in features_chosen for element in sublist]

In [ ]:
features_chosen_dict = {i:features_chosen.count(i) for i in features_chosen}

In [ ]:
features_chosen_enet = pd.DataFrame({
    'feature': features_chosen_dict.keys(),
    'appeared': features_chosen_dict.values()
})
features_chosen_enet = (features_chosen_enet
                        .sort_values('appeared', ascending=False)
                        .reset_index(drop=True))

## Linear Regression for HousePricePredictor

### Features Selection

#### Correlation Matrix

In [ ]:
corr_df = train[numeric + ordinal + [target]].corr()

In [ ]:
fig = plot_corr_matrix(
    data=corr_df, target=target, num_features=10,
    width=0.7, height=0.3, annot=6.5, labelsize=6.5,
    linecolor=theme, full=True, abs_results=True, df=False, df_limit=None)
plt.show()

savefig('corr_matrix_linear', save_img, dpi=125)

In [ ]:
features_chosen_enet.head(10)

In [ ]:
enet_summary.head(15)

In [ ]:
print(features_chosen_enet['feature'].tolist())

In [ ]:
print(categorical)

In [ ]:
features_linear = [
    'lg_flrsfmean', 'overallqual', 'houseage',
    'lg_lotarea', 'overallcond', 'bsmtqual',
    'garagecars', 'kitchenqual', 'exterqual',  
]

In [ ]:
len(features_linear)

In [ ]:
data = train[features_linear + [target]].copy()

In [ ]:
data_raw = data.copy()

In [ ]:
data_cv = train_cv[features_linear + [target]].copy()

### Cut outliers

In [ ]:
data.shape

In [ ]:
sns.histplot(data[target], bins=42, alpha=1);

In [ ]:
target_trim = OutlierTrimmer(
    capping_method='iqr',
    tail='both',
    fold=1.5,
    variables=target)

In [ ]:
target_trim.fit(data)

In [ ]:
data = target_trim.transform(data)

In [ ]:
sns.histplot(data[target], bins=42, alpha=1);

In [ ]:
data.shape

#### LINEAR REGRESSION

In [ ]:
data.head()

In [ ]:
data, formula = lr_model_data_formula(
    data=data, target='price', predictors=features_linear)

In [ ]:
data.shape

In [ ]:
formula

In [ ]:
# cov_type='HC3' - for robust confidence intervals (in case of heteroscedasticity)
lr = smf.ols(formula=formula, data=data).fit(cov_type='HC3')

In [ ]:
lr.summary()

In [ ]:
y_pred_lr = lr.predict(data[features_linear])

In [ ]:
rmse_lr = mean_squared_error(data[target], y_pred_lr, squared=False)

In [ ]:
rmse_lr

In [ ]:
f = plot_lr_coef(lr, figsize=(2.5, 3.07))

In [ ]:
fig = plot_regression_diagnostics(
    model=lr, 
    data=data,
    target=target,
    figsize=(10, 9)
)
fig.suptitle('Regression diagnostic plots', x=0.210, y=0.908, fontsize=10)
plt.show()

savefig('regression_diagnostics', save_img)

In [ ]:
regression_diagnostics(lr, data[features_linear], alpha=0.05)

In [ ]:
outliers_idxs = get_cooksd_outliers_idxs(model=lr, data=data)

In [ ]:
residuals_df = pd.concat(
    objs=[y_pred_lr.rename('Predicted'), lr.resid.rename('Residuals')],
    axis=1)

In [ ]:
residuals_df.shape

In [ ]:
residuals_df['Residuals'].std()

In [ ]:
residuals_trim = OutlierTrimmer(
    capping_method='iqr',
    tail='both',
    fold=1.5,
    variables='Residuals')

In [ ]:
residuals_df_trimmed = residuals_trim.fit_transform(residuals_df)
residuals_df_trimmed = residuals_df_trimmed.rename(columns={'Residuals': 'Residuals trimmed'})

In [ ]:
residuals_df_trimmed.shape

In [ ]:
test_normality(residuals_df_trimmed['Residuals trimmed'])

In [ ]:
residuals_df_trimmed['Residuals trimmed'].std()

In [ ]:
normal_dist = np.random.normal(
    loc=0, scale=residuals_df['Residuals'].std(ddof=1), size=1288)

In [ ]:
test_normality(normal_dist)

In [ ]:
kurtosis(pd.concat([
    pd.DataFrame(normal_dist, columns=['Normal distribution']),
    residuals_df['Residuals'].to_frame(),
    residuals_df_trimmed['Residuals trimmed'].to_frame()
]))
    

In [ ]:
fig = plt.figure(figsize=(8, 2.5))

sns.kdeplot(normal_dist, lw=0.01, fill=True, color=alpha_color(palette[0], 0.75), label='Normal distribution')
sns.kdeplot(residuals_df['Residuals'], color=palette[1], label='Residuals (original)')
sns.kdeplot(residuals_df_trimmed['Residuals trimmed'], color=palette[3], label='Residuals (trimmed)')

plt.legend(**legend_inline())
# plt.title('Comparison of Residual distributions with and without outliers', **title_inline)
axis_rstyle(xticks=(-1.0, 1.0, 0.5), yticks=(0, 4, 1))
plt.grid(False)
plt.xlabel(None)
plt.show()

savefig('residuals_distributions_comparing', save_img)

### Simulations and comparing with ELASTIC NET

In [ ]:
data_cv.shape

In [ ]:
data_cv.head()

#### Hyperparameters Search

In [ ]:
enet_cv_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('estimator', ElasticNet(max_iter=100000))
])

In [ ]:
enet_cv_pipeline

In [ ]:
alphas = np.logspace(-3, 3, 30)
l1_ratio = np.arange(0.1, 1.0, 0.1)
n_folds_cv = 20

X = data_cv.loc[:, data_cv.columns!=target]
y = data_cv[target]

In [ ]:
enet_params = [{
    'estimator__alpha': alphas,
    'estimator__l1_ratio': l1_ratio
}]

In [ ]:
st = stopwatch.start()
enet_cv = GridSearchCV(
    estimator=enet_cv_pipeline,
    param_grid=enet_params,
    scoring='neg_root_mean_squared_error',
    cv=n_folds_cv,
    refit=False
)
print(f'Execution time: {stopwatch.stop(st)}')

In [ ]:
enet_cv.fit(X, y)

In [ ]:
enet_cv_best_score = abs(enet_cv.best_score_)
enet_cv_best_score

In [ ]:
enet_cv_best_params = {}

for key in enet_cv.best_params_.keys():
    new_key = key.lstrip('estimator')
    new_key = new_key.lstrip('__')
    enet_cv_best_params[new_key] = enet_cv.best_params_[key]
    
enet_cv_best_params

In [ ]:
enet = ElasticNet(**enet_cv_best_params)

In [ ]:
enet

In [ ]:
data_cv[features_linear] = StandardScaler().fit_transform(data_cv[features_linear])

In [ ]:
data_cv.head()

In [ ]:
enet.fit(data_cv[features_linear], data_cv[target])

In [ ]:
enet_coef = pd.concat([
    pd.Series(enet.feature_names_in_, name='feature'),
    pd.Series(enet.coef_, name='enet')
], axis=1)

# add Intercept as first row
enet_coef.loc[-1] = ['Intercept', enet.intercept_]
enet_coef.index = enet_coef.index + 1
enet_coef = enet_coef.sort_index()

In [ ]:
enet_coef

In [ ]:
model_names = ['LR', 'ENet']
result_simulation_regressions = simulation_regressions(
    data_raw, target, lr, enet, model_names,
    sample_frac=1, n_folds=1000)

In [ ]:
result_simulation_regressions.head()

In [ ]:
result_simulation_regressions.mean()

In [ ]:
result_simulation_regressions.std()

In [ ]:
test_normality(result_simulation_regressions['LR'])

In [ ]:
test_normality(result_simulation_regressions['ENet'])

In [ ]:
fig = plot_lr_enet_comparison(result_simulation_regressions)
# save plot
savefig('simulations', save_img)

In [ ]:
lr_coef = lr.params.to_frame(name='lr')
# add column with ci
lr_coef = pd.concat([
    lr_coef,
    lr.conf_int().rename(columns={0: 'ci_min', 1: 'ci_max'})
], axis=1)
lr_coef = lr_coef.reset_index(names='feature')

In [ ]:
lr_coef

In [ ]:
comparing_df = lr_coef.merge(enet_coef, on='feature')

In [ ]:
comparing_df

In [ ]:
comparing_df_plot = comparing_df.iloc[1:].copy()
comparing_df_plot.sort_values('lr', key=abs, ascending=True, inplace=True)
comparing_df_plot.reset_index(drop=True, inplace=True)
# xticks for plot
xticks = np.arange(1, len(comparing_df_plot['feature'])+1)
# yticks for plot
yticks = comparing_df_plot.index.tolist()
yticks = yticks[::-1]
yticks = [(i+1) for i in yticks]
ylabels = comparing_df_plot['feature'].values.tolist()
ylabels = ylabels[::-1]
ylabels = [str.upper(i) for i in ylabels]
labels = comparing_df_plot['feature']
# delta for points of scatterplots
delta_coeff = 0.125

In [ ]:
# figure
fig = plt.figure(figsize=(3, 4))
# scatterplots
ax = plt.scatter(
    x=comparing_df_plot['lr'], y=xticks+delta_coeff,
    s=7, color=palette[0], alpha=0.9
)
ax = plt.scatter(
    x=comparing_df_plot['enet'],y=xticks-delta_coeff,
    s=7, color=palette[1], alpha=0.9
)
# plot errorbar
yerr = ([comparing_df_plot['lr'] - comparing_df_plot['ci_min'],
         comparing_df_plot['ci_max'] - comparing_df_plot['lr']])
plt.errorbar(
    x=comparing_df_plot['lr'], y=xticks+delta_coeff,
    xerr=yerr, fmt='none', elinewidth=1, capsize=1.2,
    capthick=1, alpha=0.9)
# plot zero line
plt.axvline(
    0, ymin=0.02, ymax=0.98, color=palette[0], lw=0.75, alpha=0.25)
# legend
plt.legend(
    labels=['LR', 'ENet'], loc='lower right', handletextpad=-0.15)
# spines
ax.axes.spines[['top', 'right']].set_visible(True)
# ticks
plt.yticks(ticks=yticks, labels=ylabels, weight='bold', fontsize=7)
# title
plt.title('Comparing LR and ENET coefficients')
# remove grid
plt.grid(False)
# save plot
savefig('coefficients_comparing', save_img)

### Save Data

In [ ]:
simulation_datasets_dict['train_enet'] = data_cv
simulation_datasets_dict['features_enet'] = features_linear

In [ ]:
estimators_dict['enet'] = enet
estimators_dict['lr'] = lr

In [ ]:
evaluation_dict['cv_enet'] = enet_cv

In [ ]:
features_dict['features_linear'] = features_linear

In [ ]:
saveit(features_dict, 'features_dict', save_files)

In [ ]:
saveit(simulation_datasets_dict, 'simulation_datasets_dict', save_files)

In [ ]:
saveit(estimators_dict, 'estimators_dict', save_files)

In [ ]:
saveit(evaluation_dict, 'evaluation_dict', save_files)

### Save Session

In [ ]:
save_session(session_name)

### Execution time

In [ ]:
print(f'Execution time: {stopwatch.stop(start)}')